# SQL Challenge 1 Notebook


Results can be improved with clarified requirements, information about the membership onboarding flow, and a more complete time-stamp. Lacking this information, I have made and recorded several assumptions about what information is actually desired.
    
Since it was requested that the results would be accessible on demand, I have structured them as views that can be accessed with very simple queries.


## Case Study Questions

1. What is the total amount each customer spent at the restaurant?

In [ ]:
CREATE VIEW customer_totals_vw AS
(
  SELECT 
  	sales.customer_id AS customer,
  	SUM(menu.price) AS total
  FROM dannys_diner.sales sales
  LEFT JOIN dannys_diner.menu menu
  ON sales.product_id = menu.product_id
  GROUP BY customer
);

SELECT * FROM customer_totals_vw;

2. How many days has each customer visited the restaurant?

In [ ]:
CREATE VIEW customer_visits_vw AS (
  SELECT customer_id, COUNT(DISTINCT order_date) AS total_visits
  FROM dannys_diner.sales
  GROUP BY customer_id
);

SELECT * FROM customer_visits_vw;

3. What was the first item from the menu purchased by each customer?
ASSUMPTION: sales are listed chronologically in the sales table from earliest to latest.

In [ ]:
CREATE VIEW first_product_vw AS
(
  SELECT DISTINCT ON (customer_id)
      sales.customer_id,
      menu.product_name
  FROM  dannys_diner.sales sales
  JOIN dannys_diner.menu menu
  ON sales.product_id = menu.product_id
  ORDER BY sales.customer_id, sales.order_date, sales.product_id ASC
);

SELECT * FROM first_product_vw;

4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [ ]:
CREATE VIEW most_popular_vw AS (
  SELECT 
      menu.product_name,
      COUNT(*) AS total_purchased
  FROM dannys_diner.sales sales
  JOIN dannys_diner.menu menu
  ON sales.product_id = menu.product_id
  GROUP BY menu.product_name
  ORDER BY total_purchased DESC
  LIMIT 1
);

SELECT * FROM most_popular_vw;

5. Which item was the most popular for each customer?

ASSUMPTION: if a customer's most-purchased item is tied across 2 or more items, choose only the first purchased item.

In [ ]:
CREATE VIEW popular_per_customer_vw AS(
  SELECT 
      most_purchased.customer_id,
      menu.product_name
  FROM
      (
        SELECT DISTINCT ON (customer_id)
          customer_id, 
          product_id,
          COUNT(*) AS product_count
        FROM dannys_diner.sales
        GROUP BY customer_id, product_id
        ORDER BY customer_id
      ) most_purchased
  JOIN dannys_diner.menu menu
  ON most_purchased.product_id = menu.product_id
);

SELECT * FROM popular_per_customer_vw;

6. Which item was purchased first by the customer after they became a member?

ASSUMPTION: ignore customers who did not become members.

ASSUMPTION: ignore items purchased on the date the customer joined.

In [ ]:
CREATE VIEW member_first_purchase AS (
  SELECT DISTINCT ON (members.customer_id)
      members.customer_id,
      menu.product_name
  FROM dannys_diner.members members
  JOIN dannys_diner.sales sales
  ON members.customer_id = sales.customer_id
  JOIN dannys_diner.menu menu
  ON sales.product_id = menu.product_id
  WHERE sales.order_date > members.join_date);

SELECT * FROM member_first_purchase;

7. Which item was purchased just before the customer became a member?

ASSUMPTION: include item purchased on the date the member joined.

In [ ]:
CREATE VIEW trigger_purchase_vw AS (
  SELECT DISTINCT ON (members.customer_id)
	members.customer_id,
	menu.product_name
  FROM dannys_diner.members members
  JOIN dannys_diner.sales sales
  ON members.customer_id = sales.customer_id
  JOIN dannys_diner.menu menu
  ON sales.product_id = menu.product_id
  WHERE sales.order_date <= members.join_date
  ORDER BY members.customer_id, sales.order_date DESC);
  
SELECT * FROM trigger_purchase_vw;

8. What is the total items and amount spent for each member before they became a member?
ASSUMPTION: ignore customers who are not yet members

In [ ]:
CREATE VIEW member_priors_vw AS (
  SELECT
    members.customer_id,
    COUNT(*) AS number_items,
    SUM(menu.price) AS total_spent
  FROM dannys_diner.members members
  JOIN dannys_diner.sales sales
  ON members.customer_id = sales.customer_id
  JOIN dannys_diner.menu menu
  ON sales.product_id = menu.product_id
  WHERE members.join_date >= sales.order_date
  GROUP BY members.customer_id);

SELECT * FROM member_priors_vw;

9.  If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [ ]:
CREATE VIEW customer_pts_vw AS(
  SELECT
  	sales.customer_id AS customer,
  	SUM(
      CASE
  		WHEN sales.product_id = 1 THEN menu.price * 20
  		ELSE menu.price *10
      END) AS total_points
  FROM dannys_diner.sales sales
  JOIN dannys_diner.menu menu
  ON sales.product_id = menu.product_id
  GROUP BY customer
);

SELECT * FROM customer_pts_vw;

10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

ASSUMPTION: The bonus window should close at the end of the 7th day, where day 1 is the day of joining

In [ ]:
CREATE VIEW customer_pts_bonus_vw AS(
  SELECT
  	sales.customer_id AS customer,
  	SUM(
      CASE
  		WHEN sales.product_id = 1 OR sales.order_date <= (members.join_date + INTERVAL '7 days') THEN menu.price * 20
  		ELSE menu.price * 10
      END) AS total_points
  FROM dannys_diner.sales sales
  JOIN dannys_diner.menu menu
  ON sales.product_id = menu.product_id
  JOIN dannys_diner.members
  ON sales.customer_id = members.customer_id
  WHERE sales.order_date < '2021-02-01'
  GROUP BY customer
);

SELECT * FROM customer_pts_bonus_vw;